In [2]:
import os
import io
import time
import posixpath
import urllib
import urllib.parse

In [3]:
import cacherequests

Using cache directory from environment variable cacherequests_cache_dir


In [4]:
url_list_input_str = '''
https://docs.oracle.com/javame/config/cldc/ref-impl/midp2.0/jsr118/index.html
'''

lines = url_list_input_str.splitlines()
lines = map(lambda x: x.strip(), lines)
lines = filter(lambda x: len(x) > 0, lines)
lines = list(lines)
url_list = lines
len(url_list)

1

In [5]:
url_list

['https://docs.oracle.com/javame/config/cldc/ref-impl/midp2.0/jsr118/index.html']

In [7]:
log_filepath = f'{time.time_ns()}-url_list.txt'
print('log_filepath', log_filepath)

with open(log_filepath, 'wb') as outfile:
    for url_str in url_list:
        content_bs = url_str.encode('utf-8')
        outfile.write(content_bs)
        outfile.write(b'\n')
os.path.getsize(log_filepath)

log_filepath 1656405702167755700-url_list.txt


78

In [20]:
current_url = url_list[0]
current_url

'https://docs.oracle.com/javame/config/cldc/ref-impl/midp2.0/jsr118/index.html'

In [9]:
response_obj = cacherequests.wrap_requests(
    url=current_url,
    check_body_size=False,
)

type(response_obj)

dict

In [10]:
response_obj.keys()

dict_keys(['url', 'method', 'status_code', 'headers', 'content_bs'])

In [11]:
response_obj['headers']

{'Server': 'AkamaiNetStorage', 'Content-Type': 'text/html', 'ETag': '"2432d6ba7460d0128bd03516f51e9c50:1411354788"', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Cache-Control': 'max-age=20630', 'Date': 'Tue, 28 Jun 2022 08:46:25 GMT', 'Content-Length': '900', 'Connection': 'keep-alive'}

In [13]:
len(response_obj['content_bs'])

2044

In [12]:
response_obj['content_bs']

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Frameset//EN" "http://www.w3.org/TR/html4/frameset.dtd">\r\n<!--NewPage-->\r\n<HTML>\r\n<HEAD>\r\n<!-- Generated by javadoc on Thu Jan 08 08:09:22 GMT+02:00 2004-->\r\n<TITLE>\r\nMID Profile\r\n</TITLE><meta name="collection" content="api">\r\n<script>window.ohcglobal || document.write(\'<script src="/en/dcommon/js/global.js">\\x3C/script>\')</script></HEAD>\r\n<FRAMESET cols="20%,80%">\r\n<FRAMESET rows="30%,70%">\r\n<FRAME src="overview-frame.html" name="packageListFrame" title="All Packages">\r\n<FRAME src="allclasses-frame.html" name="packageFrame" title="All classes and interfaces (except non-static nested types)">\r\n</FRAMESET>\r\n<FRAME src="overview-summary.html" name="classFrame" title="Package, class and interface descriptions">\r\n<NOFRAMES>\r\n<H2>\r\nFrame Alert</H2>\r\n\r\n<P>\r\nThis document is designed to be viewed using the frames feature. If you see this message, you are using a non-frame-capable web client.\r\n<BR>\r\n

In [14]:
import bs4

In [15]:
content_str = response_obj['content_bs'].decode('utf-8')
len(content_str)

2044

In [16]:
content_str

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Frameset//EN" "http://www.w3.org/TR/html4/frameset.dtd">\r\n<!--NewPage-->\r\n<HTML>\r\n<HEAD>\r\n<!-- Generated by javadoc on Thu Jan 08 08:09:22 GMT+02:00 2004-->\r\n<TITLE>\r\nMID Profile\r\n</TITLE><meta name="collection" content="api">\r\n<script>window.ohcglobal || document.write(\'<script src="/en/dcommon/js/global.js">\\x3C/script>\')</script></HEAD>\r\n<FRAMESET cols="20%,80%">\r\n<FRAMESET rows="30%,70%">\r\n<FRAME src="overview-frame.html" name="packageListFrame" title="All Packages">\r\n<FRAME src="allclasses-frame.html" name="packageFrame" title="All classes and interfaces (except non-static nested types)">\r\n</FRAMESET>\r\n<FRAME src="overview-summary.html" name="classFrame" title="Package, class and interface descriptions">\r\n<NOFRAMES>\r\n<H2>\r\nFrame Alert</H2>\r\n\r\n<P>\r\nThis document is designed to be viewed using the frames feature. If you see this message, you are using a non-frame-capable web client.\r\n<BR>\r\nL

In [17]:
soup = bs4.BeautifulSoup(content_str)
type(soup)

bs4.BeautifulSoup

In [18]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Frameset//EN" "http://www.w3.org/TR/html4/frameset.dtd">

<!--NewPage-->
<html>
<head>
<!-- Generated by javadoc on Thu Jan 08 08:09:22 GMT+02:00 2004-->
<title>
MID Profile
</title><meta content="api" name="collection"/>
<script>window.ohcglobal || document.write('<script src="/en/dcommon/js/global.js">\x3C/script>')</script></head>
<frameset cols="20%,80%">
<frameset rows="30%,70%">
<frame name="packageListFrame" src="overview-frame.html" title="All Packages"/>
<frame name="packageFrame" src="allclasses-frame.html" title="All classes and interfaces (except non-static nested types)"/>
</frameset>
<frame name="classFrame" src="overview-summary.html" title="Package, class and interface descriptions"/>
<noframes>
<h2>
Frame Alert</h2>
<p>
This document is designed to be viewed using the frames feature. If you see this message, you are using a non-frame-capable web client.
<br/>
Link to<a href="overview-summary.html">Non-frame version.</a>
</p>

In [19]:
frame_element_list = soup.select('frame')
frame_element_list

[<frame name="packageListFrame" src="overview-frame.html" title="All Packages"/>,
 <frame name="packageFrame" src="allclasses-frame.html" title="All classes and interfaces (except non-static nested types)"/>,
 <frame name="classFrame" src="overview-summary.html" title="Package, class and interface descriptions"/>]

In [21]:
frame_source_list = []
for frame_element in frame_element_list:
    try:
        frame_source_list.append(frame_element.attrs['src'])
    except Exception as ex:
        print(ex)
frame_source_list

['overview-frame.html', 'allclasses-frame.html', 'overview-summary.html']

In [22]:
frame_source = frame_source_list[0]
frame_source

'overview-frame.html'

In [24]:
urllib.parse.urljoin(current_url, frame_source)

'https://docs.oracle.com/javame/config/cldc/ref-impl/midp2.0/jsr118/overview-frame.html'

In [25]:
urllib.parse.urlsplit(current_url)

SplitResult(scheme='https', netloc='docs.oracle.com', path='/javame/config/cldc/ref-impl/midp2.0/jsr118/index.html', query='', fragment='')

In [26]:
split_result = urllib.parse.urlsplit(current_url)
split_result

SplitResult(scheme='https', netloc='docs.oracle.com', path='/javame/config/cldc/ref-impl/midp2.0/jsr118/index.html', query='', fragment='')

In [27]:
urllib.parse.urlunsplit(split_result[:3])

ValueError: not enough values to unpack (expected 6, got 4)